## Observable Trends:

* The Percentage of Overall Passing is clearly higher in Charter Schools than in District Schools.
* In all the schools students performed better in Reading than in Math
* There is a negative co-relation between Per Student Budget of School Spending and Overall Passing rate!


In [22]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_csv_path = "Resources/schools_complete.csv"
student_csv_path = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_csv_path)
student_data = pd.read_csv(student_csv_path)

In [4]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how = "left", on = ["school_name", "school_name"])

In [5]:
# For my info of columns
school_data_complete.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635


In [6]:
# For my info of columns
school_data.head(1)

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635


In [7]:
# For my info Columns
student_data.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79


## District Summary

In [9]:
# --Calculate pass % for math------

# Criterion to pass math
to_pass_math = student_data[student_data.math_score > 70]  

# Percentage of Students meeting that
math_pass = ( to_pass_math['math_score'].count() / student_data['student_name'].count() ) * 100 
 

#----- Calculate pass % for reading--------
     
# Criterion to pass reading 
to_pass_reading = student_data[student_data.reading_score > 70]  
    
# Percentage of Students meeting that
reading_pass =( to_pass_reading['reading_score'].count() / student_data['student_name'].count() ) * 100  

#----- Create a dictionary of Key : Column and Row : Values ----

Dist_Dict = {'Total Schools': [school_data['school_name'].count()], 
     'Total Students': [student_data['student_name'].count()], 
     'Total Budget':[school_data['budget'].sum()], 
     'Average Math score':[round (student_data['math_score'].mean(),2)], 
     'Average Reading score':[round (student_data['reading_score'].mean(),2)],
     '% Passing Math':[round (math_pass,2)],
     '% Passing Reading':[round (reading_pass, 2)],
     '% Overall Passing Rate':[ round((math_pass + reading_pass)/2, 2)]
    }

# Convert that to a dataframe
Dist_Summary_df = pd.DataFrame(Dist_Dict)

# access all those cells using the dataframe.loc

Dist_Summary_df = Dist_Summary_df.loc[:,['Total Schools','Total Students','Total Budget','Average Math score',
                                       'Average Reading score','% Passing Math','% Passing Reading', '% Overall Passing Rate']]

# Format the Display

Dist_Summary_df['Total Budget'] = Dist_Summary_df['Total Budget'].map("${:,.0f}".format)
Dist_Summary_df['Total Students'] = Dist_Summary_df['Total Students'].map("{:,.0f}".format)
Dist_Summary_df['% Passing Math'] = Dist_Summary_df['% Passing Math'].map("{:,}%".format)
Dist_Summary_df['% Passing Reading'] = Dist_Summary_df['% Passing Reading'].map("{:,}%".format)
Dist_Summary_df['% Overall Passing Rate'] = Dist_Summary_df['% Overall Passing Rate'].map("{:,}%".format)



# Show!

Dist_Summary_df


,Total Schools,Total Students,Total Budget,Average Math score,Average Reading score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,72.39%,82.97%,77.68%


## School Summary
 
* Create an overview table that summarizes key metrics about each school, including:
* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* Overall Passing Rate (Average of the above two)


* Create a dataframe to hold the above result

In [13]:
#-------------------------- School metrics  ----------------------------------

# Copy school_data for manipulations

copy_school = school_data.copy()
  
# Convert back to a dataframe 

school_metrics = pd.DataFrame(copy_school)

# Rename the Columns 

school_metrics.rename(columns = { 'type'   : 'School Type',
                                  'size'   : 'Total Students',
                                  'budget' : 'Total School Budget',
                                }, inplace = True)
    
# Calculate the Per-Student Budget and add it to the dataframe

school_metrics['Per Student Budget'] = school_metrics['Total School Budget'] / school_metrics['Total Students']

# ------------------------ Student Metrics ------------------------------------- 

# Calculate math score and reading score by school 

copied_for_average = student_data.groupby(['school_name'])

grouped_average_scores = pd.DataFrame ( round (copied_for_average.mean(),2))

grouped_average_scores = grouped_average_scores.loc[:,['reading_score','math_score']]

grouped_average_scores


# Calculate passing percentages for math and reading by school

# -------- Reading------

# Copy data 
reading_pass = student_data.copy()  

# locate cells of school_name and respective reading scores  
reading_pass = reading_pass.loc[:,['school_name','reading_score']]

# pick only those reading scores greater than 70 for pass eligibility
reading_pass['reading_score'] = reading_pass.loc[reading_pass['reading_score'] > 70]

# now group reading scores by school
reading_pass = reading_pass.groupby('school_name')

# calculate the total reading scores by school for later use
reading_pass = pd.DataFrame(reading_pass.count())

# ------- Math -----------
# Copy data
math_pass = student_data.copy()  

# locate cells of school_name and respective math scores  
math_pass = math_pass.loc[:,['school_name','math_score']]

# pick only those math scores greater than 70 for pass eligibility
math_pass['math_score'] = math_pass.loc[math_pass['math_score'] > 70]

# now group math scores by school
math_pass = math_pass.groupby('school_name')

# calculate the total math scores by school for later use
math_pass = pd.DataFrame(math_pass.count())
math_pass

#---- Overall Pass ------
# merge the reading and math pass scores
overall_pass = reading_pass.join(math_pass)

# Convert to Dataframe
overall_pass = pd.DataFrame(overall_pass)

# Rename since there is a column name clash between Individual scores and Overall Scores Column
overall_pass.rename(columns = {
                               'reading_score':'Overall_ReadScores',
                               'math_score': 'Overall_MathScores'
                               },inplace = True)


# Join the school metrics and Student metrics
school_summary = school_metrics.join(grouped_average_scores, on ='school_name')

# Convert it to a DataFrame
School_Summary_df = pd.DataFrame(school_summary)

# incorporate the overall scores to the dataframe
School_Summary_df = School_Summary_df.join( overall_pass, on = 'school_name')

# Calculate pass percentage for math,reading and overall
School_Summary_df['% Passing Math'] = round(School_Summary_df['Overall_MathScores'] / School_Summary_df['Total Students']*100,2)
School_Summary_df['% Passing Reading'] = round(School_Summary_df['Overall_ReadScores'] / School_Summary_df['Total Students']*100,2)
School_Summary_df['% Overall Passing Rate'] = (School_Summary_df['% Passing Math'] + School_Summary_df['% Passing Reading']) / 2

# Rename Columns 
School_Summary_df.rename(columns = { 'school_name'   : 'School Name',
                                     'reading_score' : 'Average Reading Score',
                                     'math_score'    : 'Average Math Score'
                                   }, inplace = True)
# Drop columns not to show
School_Summary_df = School_Summary_df.drop(['School ID','Overall_ReadScores','Overall_MathScores'], axis = 1)

# Show !
School_Summary_df


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,81.18,76.63,63.32,78.81,71.065
1,Figueroa High School,District,2949,1884411,639.0,81.16,76.71,63.75,78.43,71.090
2,Shelton High School,Charter,1761,1056600,600.0,83.73,83.36,89.89,92.62,91.255
3,Hernandez High School,District,4635,3022020,652.0,80.93,77.29,64.75,78.19,71.470
4,Griffin High School,Charter,1468,917500,625.0,83.82,83.35,89.71,93.39,91.550
5,Wilson High School,Charter,2283,1319574,578.0,83.99,83.27,90.93,93.25,92.090
6,Cabrera High School,Charter,1858,1081356,582.0,83.98,83.06,89.56,93.86,91.710
7,Bailey High School,District,4976,3124928,628.0,81.03,77.05,64.63,79.30,71.965
8,Holden High School,Charter,427,248087,581.0,83.81,83.80,90.63,92.74,91.685
9,Pena High School,Charter,962,585858,609.0,84.04,83.84,91.68,92.20,91.940


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
School_Summary_df.nlargest(5,'% Overall Passing Rate')

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
5,Wilson High School,Charter,2283,1319574,578.0,83.99,83.27,90.93,93.25,92.090
9,Pena High School,Charter,962,585858,609.0,84.04,83.84,91.68,92.20,91.940
10,Wright High School,Charter,1800,1049400,583.0,83.96,83.68,90.28,93.44,91.860
6,Cabrera High School,Charter,1858,1081356,582.0,83.98,83.06,89.56,93.86,91.710
8,Holden High School,Charter,427,248087,581.0,83.81,83.80,90.63,92.74,91.685


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
School_Summary_df.nsmallest(5,'% Overall Passing Rate')

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,80.74,76.84,64.07,77.74,70.905
0,Huang High School,District,2917,1910635,655.0,81.18,76.63,63.32,78.81,71.065
12,Johnson High School,District,4761,3094650,650.0,80.97,77.07,63.85,78.28,71.065
1,Figueroa High School,District,2949,1884411,639.0,81.16,76.71,63.75,78.43,71.090
3,Hernandez High School,District,4635,3022020,652.0,80.93,77.29,64.75,78.19,71.470


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
math_by_grade = student_data.copy()

# choose index, columns,values and  map it accross using the .pivot_table method from the pandas library
math_by_grade = pd.pivot_table( math_by_grade , index = ['school_name'] , columns = 'grade' , values = 'math_score').reset_index()

#Show!
math_by_grade

grade,school_name,10th,11th,12th,9th
0,Bailey High School,76.996772,77.515588,76.492218,77.083676
1,Cabrera High School,83.154506,82.765560,83.277487,83.094697
2,Figueroa High School,76.539974,76.884344,77.151369,76.403037
3,Ford High School,77.672316,76.918058,76.179963,77.361345
4,Griffin High School,84.229064,83.842105,83.356164,82.044010
5,Hernandez High School,77.337408,77.136029,77.186567,77.438495
6,Holden High School,83.429825,85.000000,82.855422,83.787402
7,Huang High School,75.908735,76.446602,77.225641,77.027251
8,Johnson High School,76.691117,77.491653,76.863248,77.187857
9,Pena High School,83.372000,84.328125,84.121547,83.625455


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [19]:
reading_by_grade = student_data.copy()

# # choose index, columns,values and  map it accross using the .pivot_table method from the pandas library
reading_by_grade = pd.pivot_table( reading_by_grade , index = ['school_name'] , columns = 'grade' , values = 'reading_score').reset_index()

# Show!
reading_by_grade

grade,school_name,10th,11th,12th,9th
0,Bailey High School,80.907183,80.945643,80.912451,81.303155
1,Cabrera High School,84.253219,83.788382,84.287958,83.676136
2,Figueroa High School,81.408912,80.640339,81.384863,81.198598
3,Ford High School,81.262712,80.403642,80.662338,80.632653
4,Griffin High School,83.706897,84.288089,84.013699,83.369193
5,Hernandez High School,80.660147,81.396140,80.857143,80.866860
6,Holden High School,83.324561,83.815534,84.698795,83.677165
7,Huang High School,81.512386,81.417476,80.305983,81.290284
8,Johnson High School,80.773431,80.616027,81.227564,81.260714
9,Pena High School,83.612000,84.335938,84.591160,83.807273


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
school_spending = School_Summary_df.copy()

# make use of np.array for grouping
lable_spending = np.array(["<$585", "$585-615", "$615-645", "$645-675"])

# Use pd.qcut function to discretize scores into respective groups of Per Student Budget  
school_spending['Per Student Budget'] = pd.qcut(school_spending['Per Student Budget'], 4, labels = lable_spending )

# group by Per Student Budget
school_spending.groupby(['Per Student Budget'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math',
                                              '% Overall Passing Rate'].mean()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.935000,83.4525,93.322500,90.350000,91.836250
$585-615,83.155000,81.9000,89.377500,83.977500,86.677500
$615-645,81.916667,78.9900,83.026667,72.676667,77.851667
$645-675,80.957500,77.0225,78.197500,64.417500,71.307500


## Scores by School Size

* Perform the same operations as above, based on school size.

In [24]:
school_size_arrange = School_Summary_df.copy()

# make use of np.array for grouping
lable_size = np.array(["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"])

# # Use pd.qcut function to discretize scores into respective groups of School Sizes   
school_size_arrange['Total Students'] = pd.qcut(school_size_arrange['Total Students'], 3, labels=lable_size )


# group by Total Students (School Size)
school_size_arrange.groupby(['Total Students'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math',
                                            '% Overall Passing Rate'].mean()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.850,83.554,92.772,90.424,91.598
Medium (1000-2000),82.772,80.748,87.374,79.968,83.671
Large (2000-5000),80.966,76.992,78.388,64.210,71.299


## Scores by School Type

* Perform the same operations as above, based on school type.

In [25]:
school_type_arrange = School_Summary_df.copy()

# group by School Type
school_type_arrange.groupby(['School Type'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math',
                                            '% Overall Passing Rate'].mean().reset_index()

,School Type,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
0,Charter,83.897500,83.472500,93.051250,90.361250,91.706250
1,District,80.965714,76.955714,78.322857,64.302857,71.312857
